In [2]:

# Example of loading a pix2pix model and using it for one-off image translation

import osr
import gdal
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import load
from numpy import expand_dims
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import geopandas as gp
from pathlib import Path
import os
from PIL import Image
#from skimage.morphology import skeletonize
#from skimage.io import imread
#from skimage.io import imsave

# load an image
def load_image(myfile, size=(256,256)):
    # load image with the preferred size
    pixels = load_img(myfile, target_size=size)
    # convert to numpy array
    pixels = img_to_array(pixels)
    # scale from [0,255] to [-1,1] las gans trabajan centradas en el origen
    pixels = (pixels - 127.5) / 127.5
    # reshape to 1 sample
    pixels = expand_dims(pixels, 0)
    return pixels


Using TensorFlow backend.


In [3]:

# Nombre de los folders

raster_in_folder_name = 'raster_in'
data_folder_name = 'raster_to_shape'
shape_out_folder_name = 'shape_out'
raster_out_folder_name = 'raster_out'

rasterin_folder = "C:/Users/john/Downloads/AutoMap/datos/"
data_folder = "C:/Users/john/Downloads/AutoMap/datos/"
shapeout_folder = "C:/Users/john/Downloads/AutoMap/outputs/"
raster_out_folder = "C:/Users/john/Downloads/AutoMap/outputs/"

segmenter_path = "C:/Users/john/Downloads/AutoMap/model"
tiles_path = Path(data_folder + '/tiles')  # from Orthomosaics
masks_path = Path(data_folder + '/masks')  # from roads in raster

tile_width = 256
tile_height = 256

type = 'roads'
#modelname = 'model_127700_atributed_roads.h5'
modelname = 'model_030000.h5'
model = load_model(segmenter_path+'/'+modelname)


ValueError: Unknown layer: Functional

In [ ]:

for rasterin in os.listdir(rasterin_folder):
    os.system('venv/bin/gdal_retile.py -ps ' + str(tile_width) + ' ' 
              + str(tile_height) + ' -targetDir ' + data_folder 
              + '/tiles ' + rasterin_folder + '/' + rasterin)

    for img_name in os.listdir(data_folder + '/tiles'):
        msk_name = 'mask_' + img_name
        png_img_name = img_name.replace('.tif','.png')
        png_msk_name = msk_name.replace('.tif','.png')
        
        source_raster = gdal.Open(data_folder + '/tiles/' + img_name)
        srs_geotrans = source_raster.GetGeoTransform()
        srs_proj = source_raster.GetProjection()

        raster_path = tiles_path/img_name
        img = load_image(raster_path)
        res_img = model.predict(img)
        # Imagen resultante se vuelve al intervalo [0,1]
        res_img = (res_img + 1) / 2.0
        
        #plt.imshow(res_img[0])
        #plt.axis('off')
        #plt.show()

        #print(masks_path/img_name)

        mpimg.imsave(tiles_path/png_img_name, res_img[0])
        mpimg.imsave(masks_path/png_msk_name, res_img[0])

        rs_res_img = Image.open(data_folder + '/masks/' + png_img_name)
        rs_res_img = rs_res_img.resize((tile_width, tile_height))
        rs_res_img.save(data_folder + '/masks/' + img_name)

        rs_res_mask = Image.open(data_folder + '/masks/' + png_msk_name)
        rs_res_mask = rs_res_mask.resize((tile_width, tile_height))
        rs_res_mask = rs_res_mask.convert('1',colors=1)
        rs_res_mask.save(data_folder + '/masks/' + msk_name)

        if(type == 'roads'):
            image_to_sk = imread(data_folder + '/masks/' + msk_name)
            rs_res_mask = skeletonize(image_to_sk)
            imsave(data_folder + '/masks/' + msk_name,rs_res_mask)

        dataset = gdal.Open(data_folder + '/masks/' + msk_name)

        width = dataset.RasterXSize
        height = dataset.RasterYSize
        datas = dataset.ReadAsArray(0,0,width,height)
        driver = gdal.GetDriverByName("GTiff")
        tods = driver.Create(data_folder + '/masks/srs_' + img_name,width,height,3,
                             options=["INTERLEAVE=PIXEL"])
        tods.SetProjection(srs_proj)
        tods.SetGeoTransform(srs_geotrans)
        tods.WriteRaster(0,0,width,height,datas.tostring(),width,height,band_list=[1])
        #tods = driver.Create(data_folder + '/masks/srs_' + img_name,width,height,3,
                             options=["INTERLEAVE=PIXEL"])
        #tods.SetProjection(srs_proj)
        #tods.SetGeoTransform(srs_geotrans)
        #tods.WriteRaster(0,0,width,height,datas.tostring(),width,height,band_list=[1])

        rasterfn = data_folder + '/masks/srs_' + img_name
        maskfn = data_folder + '/masks/' + msk_name
        outSHPfn = data_folder + '/shapes/' + img_name + '.shp'
        os.system('venv/bin/gdal_polygonize.py -8 -f "ESRI Shapefile" -mask ' + 
                  maskfn + ' ' + rasterfn + ' ' + outSHPfn)

    inSHPfn = data_folder + '/shapes/'
    outSHPfn = shapeout_folder + '/' + rasterin.split('.')[0] + '_' + type
    if os.path.exists(outSHPfn + '.shp'):
        os.remove(outSHPfn + '.shp')
    if os.path.exists(outSHPfn + '.dbf'):
        os.remove(outSHPfn + '.dbf')
    if os.path.exists(outSHPfn + '.prj'):
        os.remove(outSHPfn + '.prj')
    if os.path.exists(outSHPfn + '.cpg'):
        os.remove(outSHPfn + '.cpg')
    if os.path.exists(outSHPfn + '.shx'):
        os.remove(outSHPfn + '.shx')

    os.system('venv/bin/ogrmerge.py -single -o ' + outSHPfn + '.shp' + ' ' + inSHPfn + '*.shp')

    for file in os.listdir(data_folder + '/tiles'):
        os.remove(data_folder + '/tiles/' + file)

    for file in os.listdir(data_folder + '/shapes'):
        os.remove(data_folder + '/shapes/' + file)

    for file in os.listdir(data_folder + '/masks'):
        os.remove(data_folder + '/masks/' + file)

    gdf = gp.read_file(outSHPfn + '.shp')
    gpproj = gdf.crs
    gdf['x'] = gdf.centroid.map(lambda p: p.x)
    gdf['y'] = gdf.centroid.map(lambda p: p.y)
    gdf['type']= type
    del gdf['DN']
    gdf.insert(0, 'ID', range(1, 1 + len(gdf)))
    gdf = gdf.to_crs('EPSG:3116')
    gdf['area']= gdf['geometry'].area
    gdf = gdf.to_crs(gpproj)
    gdf.to_file(outSHPfn + '.shp')